In [1]:
from ipynb.fs.full.flights_models_v5 import *

C:\Users\duart\AppData\Local\conda\conda\envs\testEnv\lib\site-packages\IPython\core\interactiveshell.py:3242: DtypeWarning: Columns (48,69,76,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [2]:
data.head()

,DayOfWeek,ArrDel15,CRSDepTimeHour,dummyOriginATL,dummyOriginORD,dummyOriginDFW,dummyOriginDEN,dummyOriginCLT,dummyOriginLAX,dummyOriginIAH,...,Airline_EV,Airline_F9,Airline_MQ,Airline_NK,Airline_OH,Airline_OO,Airline_UA,Airline_WN,Airline_YV,Airline_YX
882,5,0.0,8,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
950,5,0.0,18,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2101,5,0.0,9,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2105,5,0.0,14,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2200,5,0.0,20,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [112]:
# date / time, probability 

def recommend(departure_time, origin, destination, airline = None, threshold = 2):
        """
        departure_time: dayofmonth/month/year hh:mm, str
        origin: three letter code for origin airport, str
        destination: three letter code for destination airport, str
        airline: two letter code for airline, str
        threshold: time interval the customer wants the flight recommendations, int
        """
                
        airlines = ['9E','AA', 'AS', 'DL', 'EV', 'F9','MQ', 'NK', 'OH', 'OO', 'UA','WN', 'YV', 'YX']
        
        a = []
        
        # define time intervals according to the threshold define by the customer

        max_hour = int(departure_time[11:13]) + threshold
        min_hour = int(departure_time[11:13]) - threshold

        recommendations = pd.DataFrame(columns = ["Date", "Time", "Airline", "Origin", "Destination", "Probability of Delay"])
            
        for x in np.arange(min_hour, max_hour + 1):
            
            # if the customer does not enter an airline, we will show results of all airlines
            
            if airline == None:
                
                # loop for all the airlines
                
                for b in airlines:
                    prediction = predict_delay(departure_time[:11] + str(x) + departure_time[13:], b, origin, destination)
                    probability = 1 - prediction
                
                    #append row to the dataframe
                    
                    to_append = [departure_time[:10], str(x) + departure_time[13:], b, origin, destination, probability]
                    series = pd.Series(to_append, index = recommendations.columns)
                    recommendations = recommendations.append(series, ignore_index = True)
                    
            # if the customer does enter an airline, we will only show results of that airline
            
            else:
                prediction = predict_delay(departure_time[:11] + str(x) + departure_time[13:], airline, origin, destination)
                probability = 1 - prediction
                
                #append row to the dataframe
                
                to_append = [departure_time[:10], str(x) + departure_time[13:], airline, origin, destination, probability]
                series = pd.Series(to_append, index = recommendations.columns)
                recommendations = recommendations.append(series, ignore_index = True)
        
        recommendations.sort_values(by = "Probability of Delay", ascending = True, inplace = True)
        recommendations['Probability of Delay'] = pd.Series(["{0:.2f}%".format(val * 100) for val in recommendations['Probability of Delay']], index = recommendations.index)
        
        return recommendations.head(10)

In [113]:
recommend("15/11/2020 10:30", "JFK", "ATL", "AA")

,Date,Time,Airline,Origin,Destination,Probability of Delay
4,15/11/2020,12:30,AA,JFK,ATL,2.53%
3,15/11/2020,11:30,AA,JFK,ATL,10.05%
0,15/11/2020,8:30,AA,JFK,ATL,11.12%
2,15/11/2020,10:30,AA,JFK,ATL,13.63%
1,15/11/2020,9:30,AA,JFK,ATL,13.80%
